## Edge betweennessによるコミュニティ抽出

In [36]:
import plotly.graph_objects as go
import networkx as nx

import numpy as np
import pandas as pd
import random

from matplotlib import pyplot as plt

In [86]:
#GRAPH_INPUT = '../s7_mini_seed7427.txt'
GRAPH_INPUT = '../s1_seed7552.txt'

with open(GRAPH_INPUT, 'r') as f:
    graph_def_str = f.readline().replace('\n', '')
    
graph_def = list(map(int, graph_def_str.split()))    

In [87]:
N, M, D, K = graph_def[:4]
graph_def = graph_def[4:]

edge_info_list = {}
edge_list = []
nodes_to_edge = {}

for i in range(M):
    u = graph_def.pop(0)
    v = graph_def.pop(0)
    w = graph_def.pop(0)
    
    if u not in edge_info_list.keys():
        edge_info_list[u] = []
        
    if v not in edge_info_list.keys():
        edge_info_list[v] = []

    edge_info_list[u].append((i,v,w))
    edge_info_list[v].append((i,u,w))
    
    edge_list.append((u, v, w))
    nodes_to_edge[(u, v)] = i
    nodes_to_edge[(v, u)] = i

pos_list = {}

for i in range(N):
    u = i + 1
    x = graph_def.pop(0)
    y = graph_def.pop(0)
    
    pos_list[u] = (x, y)

In [88]:
G = nx.Graph()
G.add_weighted_edges_from(edge_list)

In [89]:
def add_betweenness(u, v, edge_between, cost):
    if (u, v) not in edge_between.keys():
        edge_between[(u, v)] = 0

    if (v, u) not in edge_between.keys():
        edge_between[(v, u)] = 0
        
    edge_between[(u, v)] += cost
    edge_between[(v, u)] += cost
    
    return edge_between

## 最短路木

In [90]:
edge_between = {}

for s in range(1, N+1):
    orig_shortest_path = nx.shortest_path(G, source=s, weight='weight')
    
    for node in range(1, N+1):
        for i in range(1, len(orig_shortest_path[node])):
            u = orig_shortest_path[node][i-1]
            v = orig_shortest_path[node][i]

            edge_between = add_betweenness(u, v, edge_between, 1)
#            edge_between = add_betweenness(u, v, edge_between, G[u][v]['weight'])


In [91]:
fig = go.Figure()

# 幅を固定
fig.update_layout(
    width=750,
    height=750,
)

# 背景を白に変更
fig.update_layout(plot_bgcolor='white')

grid_x = []
grid_y = []

draw_node_list = set()

for node in range(1, N+1):
    x, y = pos_list[node]
    
    grid_x.append(x)
    grid_y.append(y)

label = list(map(str, draw_node_list))

fig.add_trace(
    go.Scatter(x=grid_x, y=grid_y, mode='markers+text', text=label, marker={'size': 6, 'color': '#333333'}, name='', textposition='top left')
)

drawed = set()

bet_min = min(edge_between.values())
bet_max = max(edge_between.values())

for u, v in edge_between.keys():
    if (u, v) in drawed:
        continue
        
    from_x, from_y = pos_list[u]
    to_x, to_y = pos_list[v]

    rank = (edge_between[(u, v)] - bet_min) / (bet_max - bet_min)
    rank = int(10 * rank) + 1

    line_def = {'width': rank, 'color': '#00d5ff'}
            
    fig.add_trace(
        go.Scatter(x=[from_x, to_x], y=[from_y, to_y], mode='lines', line=line_def, name='')
    )

    drawed.add((u, v))
    drawed.add((v, u))
        
fig.update_layout(showlegend=False)        

fig.show()

## FaceGroupの可視化

In [92]:
face_edge_list = [257,871,735,1508,806,641,54,760  ]
#face_edge_list = [786, 893]

In [93]:
for e in face_edge_list:
    u, v, w = edge_list[e]
    print(e, edge_between[(u, v)])

257 2104
871 1522
735 1198
1508 1088
806 894
641 640
54 540
760 2


In [94]:
fig = go.Figure()

# 幅を固定
fig.update_layout(
    width=750,
    height=750,
)

# 背景を白に変更
fig.update_layout(plot_bgcolor='white')

grid_x = []
grid_y = []

draw_node_list = set()

for node in range(1, N+1):
    x, y = pos_list[node]
    
    grid_x.append(x)
    grid_y.append(y)

label = list(map(str, draw_node_list))

fig.add_trace(
    go.Scatter(x=grid_x, y=grid_y, mode='markers+text', text=label, marker={'size': 6, 'color': '#333333'}, name='', textposition='top left')
)

drawed = set()

bet_min = min(edge_between.values())
bet_max = max(edge_between.values())

for u, v in edge_between.keys():
    if (u, v) in drawed:
        continue    
    
    from_x, from_y = pos_list[u]
    to_x, to_y = pos_list[v]

    rank = (edge_between[(u, v)] - bet_min) / (bet_max - bet_min)
    rank = int(10 * rank) + 1

    e = nodes_to_edge[(u, v)]
    label=''
    
    if e in face_edge_list:
        line_def = {'width': rank, 'color': '#00d5ff'}
        label = str(e)
    else:
        line_def ={'width': 2, 'color': '#333333'}
            
    fig.add_trace(
        go.Scatter(x=[from_x, to_x], y=[from_y, to_y], mode='lines', line=line_def, name='')
    )
    
    if e in face_edge_list:
        fig.add_trace(
            go.Scatter(x=[(from_x + to_x)//2], y=[(from_y + to_y)//2], mode='text', text=label, name='', textposition='middle center')
        )

    drawed.add((u, v))
    drawed.add((v, u))
        
fig.update_layout(showlegend=False)        

fig.show()